In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [37]:
from factue.methods.llm_calls import load_template_list, load_template_parts
for x in load_template_list(job='persuasion', step="explain", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Guilt_by_Association"
"Doubt"
"Appeal_to_Hypocrisy"
"Questioning_the_Reputation"
"Flag_Waving"
"Appeal_to_Authority"
"Appeal_to_Popularity"
"Appeal_to_Values"
"Appeal_to_Fear-Prejudice"
"Straw_Man"
"Red_Herring"
"Whataboutism"
"Appeal_to_Pity"
"Causal_Oversimplification"
"False_Dilemma-No_Choice"
"Consequential_Oversimplification"
"False_Equivalence"
"Slogans"
"Conversation_Killer"
"Appeal_to_Time"
"Loaded_Language"
"Obfuscation-Vagueness-Confusion"
"Exaggeration-Minimisation"
"Repetition"


In [41]:
import pandas as pd
from pathlib import Path
import os
import json 
import numpy as np

root = Path("data/llm_output/persuasion/explain")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

def safe_description(data):
    if isinstance(data, (list,np.ndarray)):
        return data[-1]
    else:
        return None  # or "", depending on what you prefer
    
df['expected_output'] = df.apply(lambda row: json.dumps({
    'description': safe_description(row['explanation']),
    'verdict': bool(row['gold'])
}), axis=1)
df

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output
0,pl_eu_12_06_2024_n01.txt,0,135,PL,Wicemarszałek Włodzimierz Czarzasty:\nBardzo p...,False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1,pl_defence_22_05_2024_n01.txt,3608,3857,PL,Pan wspominał o ABCS – Integrated Battle Comma...,False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
2,pl_current_affairs_09_01_2025_n03.txt,1808,1984,PL,(Poseł Anna Gembicka: Decyzja została zmienion...,True,[Repetition],train,[],[],...,[None],[The Appeal to Hypocrisy technique was not use...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The Appeal to Hypocrisy techn..."
3,pl_abortion_11_04_2024_n08.txt,2952,3136,PL,"Czy czas najwyższy, by w końcu dopuścić\ndekry...",True,"[False_Equivalence, Loaded_Language, Whatabout...",train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
4,pl_eu_12_06_2024_n01.txt,1277,2304,PL,"Prezentowany na posiedzeniu projekt ustawy, op...",False,[],train,[],[],...,[None],[The input text discusses a proposed law aimed...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text discusses a pr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875,20231208_URW_BG_71.txt,3950,3965,BG,Има ли реплика?,False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1876,20231208_URW_BG_76.txt,1541,1938,BG,"Най-интересното е, че в т. 7 пише, цитирам, че...",False,[],train,[],[],...,[None],[The input text does not employ the Casting Do...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not emplo..."
1877,20240704_URW_BG_25.txt,4841,5237,BG,"и, предлагам това заседание, госпожо Председат...",True,"[Appeal_to_Hypocrisy, Loaded_Language]",train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."
1878,20240704_URW_BG_25.txt,1179,1237,BG,"АВ РИБАРСКИ (ПП-ДБ): Благодаря, госпожо Предсе...",False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": ""The input text does not conta..."


In [45]:
df[df["is_valid"].apply(
    lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [False]
)]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output
90,pl_abortion_11_04_2024_n03.txt,4168,5204,PL,"W latach 90., które już bardzo dobrze pamiętam...",True,"[Appeal_to_Hypocrisy, Loaded_Language, Questio...",train,[ustawa była początkiem wymazywania kobiet z d...,[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
201,SI_interpelation_interior_12.txt,4448,7979,SI,Najlepša hvala za besedo. Najprej hvala gospod...,True,"[Consequential_Oversimplification, Exaggeratio...",train,"[tisti, ki vam je pisal odgovore na našo inter...",[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
209,SI_interpelation_interior_7.txt,2598,5288,SI,"Policisti prav tako dobijo navodila, naj ne sp...",True,"[Consequential_Oversimplification, Exaggeratio...",train,[Taka navodila policistom seveda preprečujejo ...,[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
227,SI_interpelation_interior_7.txt,839,2596,SI,Naslednji primer: žena pomočnika generalnega d...,True,"[Causal_Oversimplification, Exaggeration-Minim...",train,[mora po navodilu pomočnika generalnega direkt...,[golobistov],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
234,SI_interpelation_interior_6.txt,270,7305,SI,Na eni izmed sej Odbora za notranje zadeve mi ...,True,"[Loaded_Language, Obfuscation-Vagueness-Confus...",train,"[Še posebej direktorja, tam na mojem koncu, ko...","[golobisti, janšistov, janšisti, janšistov, go...",...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781,20240704_URW_BG_30.txt,2946,2988,BG,"Нинова, за процедура.\n\nИзчакайте, господи",False,[],train,[],[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"
1782,20231208_URW_BG_72.txt,1795,2118,BG,"Говорите, че машините, които гласувахме – „Стр...",True,"[Questioning_the_Reputation, Doubt]",train,"[Ако за Вас 1956 г. е същата като 2001 г., за ...",[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
1783,20240229_URW_BG_67.txt,2786,4165,BG,"И да Ви кажа, уважаеми български граждани, че ...",True,"[Appeal_to_Hypocrisy, Questioning_the_Reputati...",train,"[И да Ви кажа, уважаеми български граждани, че...",[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
1814,20240704_URW_BG_25.txt,4080,4133,BG,оспожо Председател.\n\nЩе направя процедурно п...,False,[],train,[],[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"


In [44]:
df[df["explanation"].apply(
    lambda x: isinstance(x, (list, np.ndarray)) and list(x) == [None]
)]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output
48,pl_abortion_11_04_2024_n08.txt,290,1251,PL,Poseł Roman Fritz:\nGrupa feministek pomagając...,True,"[Loaded_Language, Slogans, Questioning_the_Rep...",train,[Grupa feministek pomagająca mordować nienarod...,"[Aborcyjny Dream Team, Aborcyjny Dream Team, m...",...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
90,pl_abortion_11_04_2024_n03.txt,4168,5204,PL,"W latach 90., które już bardzo dobrze pamiętam...",True,"[Appeal_to_Hypocrisy, Loaded_Language, Questio...",train,[ustawa była początkiem wymazywania kobiet z d...,[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
172,RU_migrants_5.txt,354,574,RU,йчас Каменск-Уральск во всех сводках новостей....,True,[Appeal_to_Fear-Prejudice],train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
194,RU_discrediting_the_west_7.txt,700,1046,RU,"Затем 22 марта был теракт в концертном зале ""К...",True,"[Guilt_by_Association, Appeal_to_Fear-Prejudice]",train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
201,SI_interpelation_interior_12.txt,4448,7979,SI,Najlepša hvala za besedo. Najprej hvala gospod...,True,"[Consequential_Oversimplification, Exaggeratio...",train,"[tisti, ki vam je pisal odgovore na našo inter...",[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,20240229_URW_BG_67.txt,2786,4165,BG,"И да Ви кажа, уважаеми български граждани, че ...",True,"[Appeal_to_Hypocrisy, Questioning_the_Reputati...",train,"[И да Ви кажа, уважаеми български граждани, че...",[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"
1798,20231208_URW_BG_76.txt,5170,5523,BG,"Това, което всъщност в момента даваме тук, е р...",True,"[Consequential_Oversimplification, Loaded_Lang...",train,"[след престъпниците, които управляват Армията ...",[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
1814,20240704_URW_BG_25.txt,4080,4133,BG,оспожо Председател.\n\nЩе направя процедурно п...,False,[],train,[],[],...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": false}"
1821,trail_raw_BG_13.txt,3446,4689,BG,"да се случи, когато се обърне колелото на ист...",True,"[Conversation_Killer, Loaded_Language, Slogans...",train,"[Вие не сте даже и такива, за които членувате ...","[руски агенти, руски агенти, гешефтари]",...,[None],[None],[None],[None],[False],data/llm_output/persuasion/explain/GPT_41/v001...,NaN,NaN,train,"{""description"": null, ""verdict"": true}"


In [39]:
SYSTEM_PART_NAME = "system"
USER_PART_NAME = "user"

def build_messages(text,expected_output,job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"):
    prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

    json_line = {
            "messages": [
                {"role": "system", "content": prompt_layout[SYSTEM_PART_NAME].format(technique_name=prompt_content['technique_name'],
                                                                                     technique_desc=prompt_content['technique_desc'],
                                                                                     input_example = prompt_content['input_example'],
                                                                                     output_example = prompt_content['output_example'])},
                {"role": "user", "content": prompt_layout[USER_PART_NAME].format(text = text)},
                {"role": "assistant", "content": expected_output}
            ]
        }
    return json_line
    # return(json.dumps(json_line, ensure_ascii=False) + "\n")


In [6]:
prompt_layout, prompt_content = load_template_parts(
        job="persuasion", step="detect", prompt_name="Name_Calling-Labeling", prompt_version="v001"
    )

In [7]:
# Write DataFrame to JSONL
with open("data/finetuning/explain2detect_pers_v1.jsonl", "w", encoding="utf-8") as file:
    for _, row in df[df.error.isna()].iterrows():
        json_line = build_messages(text = row['text'],expected_output = row['expected_output'])
        file.write(json.dumps(json_line, ensure_ascii=False) + "\n")

# errors

Unhandled exception during LLM call

In [31]:
errs_to_delete = df[df['error'].apply(lambda x: isinstance(x, np.ndarray) and 'Unhandled exception during LLM call' in x)]
errs_to_delete

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output


In [32]:
import os

# List of file paths
files_to_delete = errs_to_delete['source_file'].dropna().unique().tolist()

# Delete each file safely
for path in files_to_delete:
    if os.path.isfile(path):
        try:
            # os.remove(path)
            print(f"Deleted: {path}")
        except Exception as e:
            print(f"Error deleting {path}: {e}")
    else:
        print(f"File not found: {path}")

In [33]:
df[~df.error.isna()]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output
30,pl_abortion_11_04_2024_n08.txt,150,288,PL,Wicemarszałek Monika Wielichowska:\nSam pan wy...,True,[Questioning_the_Reputation],train,"[Sam pan wywołał reakcję pań, które znajdują s...",[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
31,pl_abortion_11_04_2024_n04.txt,1320,1701,PL,Zakaz aborcji w Polsce nie działa i wiedzą o t...,True,"[Conversation_Killer, Appeal_to_Pity, Repetition]",train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
32,pl_defence_22_05_2024_n01.txt,8339,10002,PL,"Proszę państwa, jeśli chodzi o te zakupy, to o...",True,"[Loaded_Language, Flag_Waving, Obfuscation-Vag...",train,[],[postsowiecki],...,[None],[The input text does not contain any instances...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
33,pl_abortion_11_04_2024_n03.txt,1839,2256,PL,Mam dzisiaj zaszczyt i przyjemność jako posłan...,True,"[Appeal_to_Time, Appeal_to_Pity, Appeal_to_Val...",train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
34,pl_current_affairs_09_01_2025_n01.txt,1187,2227,PL,"Poseł Maria Koc:\nDziękuję, panie marszałku.\n...",True,[Doubt],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,20240704_URW_BG_26.txt,6397,6449,BG,"ажаеми господин Министър-председател, господин...",False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
930,20240704_URW_BG_26.txt,2223,2390,BG,"о искам да задам, пък вече Вие ще прецените да...",False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
931,20240704_URW_BG_29.txt,2821,3126,BG,о държавни и правителствени ръководители на НА...,False,[],train,[],[],...,[None],[The input text does not contain any elements ...,[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,[None],[None],train,"{""description"": ""The input text does not conta..."
932,20231208_URW_BG_76.txt,5170,5523,BG,"Това, което всъщност в момента даваме тук, е р...",True,"[Consequential_Oversimplification, Loaded_Lang...",train,"[след престъпниците, които управляват Армията ...",[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"


In [34]:
df[df['raw'].apply(lambda x: isinstance(x, np.ndarray) and None in x)]

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,extra_content,explanation,illegal_value,think_content,is_valid,source_file,error,status,split,expected_output
38,pl_abortion_11_04_2024_n08.txt,290,1251,PL,Poseł Roman Fritz:\nGrupa feministek pomagając...,True,"[Loaded_Language, Slogans, Questioning_the_Rep...",train,[Grupa feministek pomagająca mordować nienarod...,"[Aborcyjny Dream Team, Aborcyjny Dream Team, m...",...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
134,RU_discrediting_the_west_7.txt,700,1046,RU,"Затем 22 марта был теракт в концертном зале ""К...",True,"[Guilt_by_Association, Appeal_to_Fear-Prejudice]",train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
198,20231208_URW_BG_75.txt,5280,5654,BG,"Така че не е непредизвикана агресията, убити с...",True,"[Loaded_Language, Appeal_to_Pity, Name_Calling...",train,[],[Новите еничари],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
219,RU_demography_1.txt,1283,1609,RU,"Русские, как народ, нация, культура, могут исч...",True,[Appeal_to_Fear-Prejudice],train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
230,RU_discrediting_the_west_7.txt,700,1046,RU,"Затем 22 марта был теракт в концертном зале ""К...",True,"[Guilt_by_Association, Appeal_to_Fear-Prejudice]",train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
274,pl_abortion_11_04_2024_n08.txt,290,1251,PL,Poseł Roman Fritz:\nGrupa feministek pomagając...,True,"[Loaded_Language, Slogans, Questioning_the_Rep...",train,[Grupa feministek pomagająca mordować nienarod...,"[Aborcyjny Dream Team, Aborcyjny Dream Team, m...",...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
369,RU_demography_1.txt,1283,1609,RU,"Русские, как народ, нация, культура, могут исч...",True,[Appeal_to_Fear-Prejudice],train,[],[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
470,20231208_URW_BG_76.txt,5170,5523,BG,"Това, което всъщност в момента даваме тук, е р...",True,"[Consequential_Oversimplification, Loaded_Lang...",train,"[след престъпниците, които управляват Армията ...",[],...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": false}"
570,pl_abortion_11_04_2024_n08.txt,290,1251,PL,Poseł Roman Fritz:\nGrupa feministek pomagając...,True,"[Loaded_Language, Slogans, Questioning_the_Rep...",train,[Grupa feministek pomagająca mordować nienarod...,"[Aborcyjny Dream Team, Aborcyjny Dream Team, m...",...,[None],[None],[None],[None],[True],data/llm_output/persuasion/explain/GPT_41/v001...,"[{'code': 'content_filter', 'innererror': {'co...",[no validation],train,"{""description"": null, ""verdict"": true}"
625,RU_demography_1.txt,1283,1609,RU,"Русские, как народ, нация, культура, могут исч...",True,[Appeal_to_Fear-Prejudi

In [ ]:
df[~df.error.isna()].value_counts('source_file')

source_file
data/llm_output/persuasion/explain/GPT_41/v001/Appeal_to_Hypocrisy/train/bg/train-bg-part_0004.parquet           10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/ru/train-ru-part_0001.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/bg/train-bg-part_0004.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/bg/train-bg-part_0007.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/bg/train-bg-part_0009.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/pl/train-pl-part_0000.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/pl/train-pl-part_0001.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-Labeling/train/pl/train-pl-part_0002.parquet         10
data/llm_output/persuasion/explain/GPT_41/v001/Name_Calling-